In [61]:
import pandas as pd
import numpy as np
import re

In [62]:
mn_or_bk = 'mn' #bk or mn
year = '1850'
#doubttype = 'not so confident' #doubt or not so confident
#doubttype = 'not so confident' #doubt or not so confident

In [63]:
#df = pd.read_csv('/Users/prajwal/Desktop/Columbia/C4SR/' + mn_or_bk + '_' + year + '/after cleaning/' + mn_or_bk + '_' + year + '_after_cleaning_' + doubttype + '4.csv')

df = pd.read_csv('/Users/prajwal/Desktop/Columbia/C4SR/tesseract output/run1/after cleaning/bk-1850-3-s-combine-clean.csv')


In [64]:

df.columns = ['index', 'Full Address', 'H House Number','H Street Name', 
              'c_street1', 'c_street2', 'n_street1', 'n_street2', 'H City',
              'H Address', 'Address 2', 'Address 2 House Number', 'Address 2 Street Name',
              'Address 2 City'
             ]

"""
df.columns = ['index', 'Full Address', 'H House Number', 
              'H Street Name', 'c_street1', 'c_street2', 'n_street1', 'n_street2', 
              'new h city', 'new h house number', 'new h street name', 'unidentified h info']

df.columns = ['index', 'n_street1', 'n_street2','c_street1', 'c_street2', 'Full Address', 
              'H House Number', 
              'H Street Name', 
              'new h house number', 'new h street name','new h city', 'unidentified h info']
"""

"\ndf.columns = ['index', 'Full Address', 'H House Number', \n              'H Street Name', 'c_street1', 'c_street2', 'n_street1', 'n_street2', \n              'new h city', 'new h house number', 'new h street name', 'unidentified h info']\n\ndf.columns = ['index', 'n_street1', 'n_street2','c_street1', 'c_street2', 'Full Address', \n              'H House Number', \n              'H Street Name', \n              'new h house number', 'new h street name','new h city', 'unidentified h info']\n"

In [65]:
df = df.drop(['index'], axis=1)

In [66]:
#df.loc[pd.isna(df['H House Number']) == True and pd.isna(df['Address 2']) == False]

In [67]:
df.head(100)

,Full Address,H House Number,H Street Name,c_street1,c_street2,n_street1,n_street2,H City,H Address,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City
0,168 HUDSON AVE,168.0,HUDSON AVE,NaN,NaN,NaN,NaN,NaN,168 HUDSON AVE,NaN,NaN,NaN,NaN
1,MASON N DEAN VANDERBILT AVE,NaN,NaN,NaN,NaN,MASON DEAN,VANDERBILT AVE,MASON N DEAN VANDERBILT AVE,MASON N DEAN VANDERBILT AVE,NaN,NaN,NaN,NaN
2,82 POPLAR,82.0,POPLAR,NaN,NaN,NaN,NaN,NaN,82 POPLAR,NaN,NaN,NaN,NaN
3,EDUCATION OF,NaN,NaN,NaN,NaN,NaN,NaN,EDUCATION OF,EDUCATION OF,NaN,NaN,NaN,NaN
4,N MERCHANT Â¥ H N UNION CT,NaN,NaN,NaN,NaN,UNION,CT,N MERCHANT Â¥ H N UNION CT,N MERCHANT Â¥ H N UNION CT,NaN,NaN,NaN,NaN
5,N PACIFIC BOERUM,NaN,NaN,NaN,NaN,PACIFIC,BOERUM,N PACIFIC BOERUM,N PACIFIC BOERUM,NaN,NaN,NaN,NaN
6,25 FRONT . [CT,25.0,FRONT . [CT,NaN,NaN,NaN,NaN,NaN,25 FRONT . [CT,NaN,NaN,NaN,NaN
7,141 N FRONT Â¥ H JOYALEMONN,141.0,N FRONT Â¥ H JOYALEMONN,NaN,NaN,NaN,NaN,NaN,141 N FRONT Â¥ H JOYALEMONN,NaN,NaN,NaN,NaN
8,KENT ALY NORTH,NaN,NaN,NaN,NaN,NaN,NaN,KENT ALY NORTH,KENT ALY NORTH,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,LABORER BOND,STATE / [HOYT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Combines data from various columns into two: hn_1 (for house number) and best_match (for street name)

In [69]:
hn_1 = {}
best_match = {}

for i in range(0,len(df)):
    hn_1[i] = df['H House Number'][i]
    best_match[i] = df['H Street Name'][i]

for i in range(0,len(df)):
#for i in range(35,36):
    #print('begin',hn_1[i],best_match[i])
    
    #handle corner cases
    if pd.isna(df['c_street1'][i]) == False or pd.isna(df['c_street2'][i]) == False:
        #print('1',i)
        try:
            best_match[i] = str(df['c_street1'][i]) + ' & ' + str(df['c_street2'][i])
        except:
            if pd.isna(df['c_street1'][i]) == False:
                best_match[i] = str(df['c_street1'])
            else:
                best_match[i] = str(df['c_street2'])
                
    #handle near cases
    elif pd.isna(df['n_street1'][i]) == False or pd.isna(df['n_street2'][i]) == False:
        #print('2',i)
        try:
            best_match[i] = str(df['n_street1'][i]) + ' & ' + str(df['n_street2'][i])
        except:
            if pd.isna(df['n_street1'][i]) == False:
                best_match[i] = str(df['n_street1'])
            else:
                best_match[i] = str(df['n_street2'])
    
    #if corner and near cases don't have any data, use H House Number and H Street Name
    elif pd.isna(hn_1[i]) == True or pd.isna(best_match[i]) == True:
        #print('3',i)
        #try:
            #hn_1[i] = df['H House Number'][i]
        #except:
            #pass
        try:
            best_match[i] = df['H City'][i]
        except:
            pass
        
        
    """
    #if everything above didn't work, use new h house number and new h street name
    elif pd.isna(hn_1[i]) == True or pd.isna(best_match[i]) == True:
        #print('4',i)
        try:
            hn_1[i] = df['new h house number'][i]
        except:
            pass
        try:
            best_match[i] = df['new h street name'][i]
        except:
            pass
    
    #if everything above still didn't work, use new h city for best_match
    elif pd.isna(df['new h city'][i]) == False and pd.isna(best_match[i]) == True:
        #print('5',i)
        best_match[i] = df['new h city'][i]
    
    # return a blank house number and Full Address for best_match if nothing above worked
    else:
        if pd.isna(hn_1[i]) == True and pd.isna(best_match[i]) == True:
            #print('6',i)
            hn_1[i] = ''
            best_match[i] = df['Full Address'][i]
            
    """
    
    """
    elif pd.isna(df['H House Number'][i]) == False or pd.isna(df['H Street Name'][i]) == False:
        print('3',i)
        try:
            hn_1[i] = df['H House Number'][i]
        except:
            pass
        try:
            best_match[i] = df['H Street Name'][i]
        except:
            pass
    
    elif pd.isna(df['new h house number'][i]) == False or pd.isna(df['new h street name'][i]) == False:
        print('4',i)
        try:
            hn_1[i] = df['new h house number'][i]
        except:
            pass
        try:
            best_match[i] = df['new h street name'][i]
        except:
            pass
    
    elif pd.isna(df['new h city'][i]) == False:
        print('5',i)
        best_match[i] = df['new h city'][i]
    
    else:
        print('6',i)
        hn_1[i] = ''
        best_match[i] = df['Full Address'][i]
    """
    #if pd.isna(df['H House Number'][i]) == True and pd.isna(df['new h house number'][i]) == True:
        #print('7',i)
        #hn_1[i] = ''
    
    if pd.isna(hn_1[i]) == True and pd.isna(best_match[i]) == True:
        #print('7',i)
        hn_1[i] = ''
        best_match[i] = df['H Address'][i]



In [70]:
df2 = pd.DataFrame([list(hn_1.values()), list(best_match.values())])

In [71]:
df2 = df2.T


In [72]:
df2.columns = ['hn_1','best_match']

In [73]:
"""
def find_number(text):
    num = re.findall(r'[0-9]+',str(text))
    return " ".join(num)
df2['hn_1']=df2['hn_1'].apply(lambda x: find_number(x))
"""

'\ndef find_number(text):\n    num = re.findall(r\'[0-9]+\',str(text))\n    return " ".join(num)\ndf2[\'hn_1\']=df2[\'hn_1\'].apply(lambda x: find_number(x))\n'

In [74]:
df2

,hn_1,best_match
0,168,HUDSON AVE
1,NaN,MASON DEAN & VANDERBILT AVE
2,82,POPLAR
3,NaN,EDUCATION OF
4,NaN,UNION & CT
5,NaN,PACIFIC & BOERUM
6,25,FRONT . [CT
7,141,N FRONT Â¥ H JOYALEMONN
8,NaN,KENT ALY NORTH
9,NaN,LABORER BOND & STATE / [HOYT


In [75]:
#df2.to_csv('/Users/prajwal/Desktop/Columbia/C4SR/' + mn_or_bk + '_' + year + '/for address builder/' + mn_or_bk + '_' + year + '_' + doubttype + '_ready4.csv')
df2.to_csv('/Users/prajwal/Desktop/Columbia/C4SR/tesseract output/run1/after cleaning/bk-1850-for-address-builder.csv')


In [77]:
#df.loc[[37400]]